# Evaluate Fisher information

Johann Brehmer, Kyle Cranmer, Felix Kling, Duccio Pappadopulo, Josh Ruderman 2018

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
% matplotlib inline
import logging
import os

import madminer.core
from madminer.fisherinformation import FisherInformation
from madminer.plotting import plot_fisher_information_contours_2d

logging.basicConfig(format='%(asctime)s  %(message)s', datefmt='%H:%M')

In [2]:
base_dir = '/Users/johannbrehmer/work/projects/madminer/diboson_mining/'
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/'

In [3]:
sample_dir = base_dir + 'data/samples/wgamma/'
card_dir = base_dir + 'cards/wgamma/'
ufo_model_dir = card_dir + 'SMWgamma_UFO'
run_card_dir = card_dir + 'run_cards/'
mg_process_dir = base_dir + 'data/mg_processes/wgamma/'
log_dir = base_dir + 'logs/wgamma/'
temp_dir = base_dir + 'data/temp'
delphes_dir = mg_dir + 'Delphes'
model_dir = base_dir + 'data/models/wgamma/'
result_dir = base_dir + 'data/results/wgamma/'

## Settings

In [4]:
weight_settings = [None, 1., 10., 100.]
weight_setting_filenames = ['', '_w1', '_w10', '_w100']

## FisherInformation instances

In [5]:
fisher_all = FisherInformation(sample_dir + 'samples.h5', debug=True)
fisher_tight = FisherInformation(sample_dir + 'samples_tight.h5', debug=True)

09:47  
09:47  ------------------------------------------------------------
09:47  |                                                          |
09:47  |  MadMiner v2018.10.30                                    |
09:47  |                                                          |
09:47  |           Johann Brehmer, Kyle Cranmer, and Felix Kling  |
09:47  |                                                          |
09:47  ------------------------------------------------------------
09:47  
09:47  Loading data from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples.h5
09:47  Found 2 parameters:
09:47     cWWW (LHA: dim6 1, maximal power in squared ME: (2,), range: (-0.1, 0.1))
09:47     cWWWtilde (LHA: dim6 2, maximal power in squared ME: (2,), range: (-0.1, 0.1))
09:47  Found 6 benchmarks:
09:47     sm: cWWW = 0.00e+00, cWWWtilde = 0.00e+00
09:47     morphing_basis_vector_1: cWWW = -9.39e-02, cWWWtilde = 0.07
09:47     morphing_basis_vector_2: cWWW = 0.

## Truth-level info

In [6]:
fisher_info_parton = fisher_all.calculate_fisher_information_full_truth(
    theta=[0.,0.],
    luminosity = 300000.
)

print(fisher_info_parton)

np.save(result_dir + 'information_parton.npy', fisher_info_parton)


09:47  Old vs new Fisher info calculation:
[[[ 0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00]]

 [[-2.77555756e-17  5.55111512e-17]
  [ 5.55111512e-17  0.00000000e+00]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  8.67361738e-19]]

 ...

 [[ 0.00000000e+00  5.55111512e-17]
  [ 5.55111512e-17  0.00000000e+00]]

 [[ 0.00000000e+00  1.11022302e-16]
  [ 1.11022302e-16  0.00000000e+00]]

 [[ 0.00000000e+00 -1.38777878e-17]
  [-1.38777878e-17 -2.77555756e-17]]]
[[[ 3.30811649e-02 -1.67268833e-02]
  [-1.67268833e-02  8.45764127e-03]]

 [[ 2.32982878e-01 -2.79324333e-01]
  [-2.79324333e-01  3.34883335e-01]]

 [[ 1.13444170e-03 -2.94706069e-03]
  [-2.94706069e-03  7.65589513e-03]]

 ...

 [[ 1.51332730e-01  3.27492972e-01]
  [ 3.27492972e-01  7.08714148e-01]]

 [[ 1.08895238e+01  6.22196468e-01]
  [ 6.22196468e-01  3.55505393e-02]]

 [[ 4.98854920e-02  1.01210211e-01]
  [ 1.01210211e-01  2.05340397e-01]]]
[[[ 3.30811649e-02 -1.67268833e-02]
  [-1.67268833e-02  8

09:48  Old vs new Fisher info calculation:
[[[ 2.22044605e-16  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  2.22044605e-16]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00]]

 ...

 [[ 0.00000000e+00 -4.44089210e-16]
  [-4.44089210e-16  0.00000000e+00]]

 [[ 0.00000000e+00 -2.77555756e-17]
  [-2.77555756e-17  0.00000000e+00]]

 [[ 8.88178420e-16  0.00000000e+00]
  [ 0.00000000e+00  2.22044605e-16]]]
[[[ 1.68209104 -6.14084259]
  [-6.14084259 22.41849392]]

 [[ 3.35853569  2.33839238]
  [ 2.33839238  1.62811399]]

 [[ 1.62939719  1.47287694]
  [ 1.47287694  1.33139205]]

 ...

 [[ 1.75245534  3.65285122]
  [ 3.65285122  7.61407248]]

 [[ 0.11943453  0.16961946]
  [ 0.16961946  0.24089149]]

 [[ 6.43415864  2.90694651]
  [ 2.90694651  1.31335556]]]
[[[ 1.68209104 -6.14084259]
  [-6.14084259 22.41849392]]

 [[ 3.35853569  2.33839238]
  [ 2.33839238  1.62811399]]

 [[ 1.62939719  1.47287694]
  [ 1.472

09:48  Old vs new Fisher info calculation:
[[[-1.11022302e-16  0.00000000e+00]
  [ 0.00000000e+00 -1.73472348e-18]]

 [[ 0.00000000e+00 -2.77555756e-17]
  [-2.77555756e-17  0.00000000e+00]]

 [[ 4.44089210e-16  0.00000000e+00]
  [ 0.00000000e+00  2.71050543e-20]]

 ...

 [[ 0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  2.22044605e-16]]

 [[ 0.00000000e+00 -1.08420217e-19]
  [-1.08420217e-19  0.00000000e+00]]

 [[-2.22044605e-16  0.00000000e+00]
  [ 0.00000000e+00 -4.44089210e-16]]]
[[[ 7.02278463e-01 -1.03146705e-01]
  [-1.03146705e-01  1.51496071e-02]]

 [[ 7.79958161e-02 -2.15586311e-01]
  [-2.15586311e-01  5.95896804e-01]]

 [[ 2.27804212e+00 -2.23476398e-02]
  [-2.23476398e-02  2.19230803e-04]]

 ...

 [[ 3.79857807e+01  7.20316074e+00]
  [ 7.20316074e+00  1.36591966e+00]]

 [[ 6.18601670e-03  7.78899232e-04]
  [ 7.78899232e-04  9.80734523e-05]]

 [[ 1.58512435e+00  2.00575531e+00]
  [ 2.00575531e+00  2.53800552e+00]]]
[[[ 7.02278463e-01 -1.03146705e-01]
  [-1.03146705e-01  1

[[65934104.21930647   129129.18318053]
 [  129129.18318053 67630934.40000063]]


In [11]:
fisher_info_parton_tight = fisher_tight.calculate_fisher_information_full_truth(
    theta=[0.,0.],
    luminosity = 300000.
)

print(fisher_info_parton_tight)

np.save(result_dir + 'information_parton_tight.npy', fisher_info_parton_tight)


[[33006561.30886249 -1465627.07389662]
 [-1465627.07389662 36806936.29824117]]


## Rate-only info

In [12]:
fisher_info_xsec = fisher_all.calculate_fisher_information_rate(
    theta=[0.,0.],
    luminosity = 300000.
)

print(fisher_info_xsec)

np.save(result_dir + 'information_xsec.npy', fisher_info_xsec)


[[ 2.24879374e+03 -6.41420859e+00]
 [-6.41420859e+00  1.82951736e-02]]


In [13]:
fisher_info_xsec_tight = fisher_tight.calculate_fisher_information_rate(
    theta=[0.,0.],
    luminosity = 300000.
)

print(fisher_info_xsec_tight)

np.save(result_dir + 'information_xsec_tight.npy', fisher_info_xsec_tight)


[[329.18596818 182.20807705]
 [182.20807705 100.85418746]]


## Full Fisher info (ML)

In [14]:
means, covs = fisher_all.calculate_fisher_information_full_detector(
    theta=[0.,0.],
    luminosity=300000.,
    model_file=model_dir + 'sally_ensemble_all',
    unweighted_x_sample_file = sample_dir + 'test/x_test.npy',
    ensemble_vote_expectation_weight=weight_settings
)

print(means[0])

for weight_filename, mean, cov in zip(weight_setting_filenames, means, covs):
    np.save(result_dir + 'information_mean_full{}.npy'.format(weight_filename), mean)
    np.save(result_dir + 'information_covariance_full{}.npy'.format(weight_filename), cov)

08:58  Loading ensemble setup from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_all/ensemble.json
08:58  Found ensemble with 10 estimators and expectations [[-0.06145189  0.01875553]
 [ 0.01819749 -0.00887818]
 [-0.03638373 -0.00889653]
 [-0.00757266  0.01537625]
 [ 0.04581081  0.06529754]
 [ 0.05060499  0.06112248]
 [-0.01120506 -0.02417562]
 [ 0.08813245 -0.00900674]
 [ 0.01593077  0.02033578]
 [ 0.02439147 -0.03597279]]
08:58  Evaluating Fisher information for 10 estimators in ensemble
08:58  Starting evaluation for estimator 1 / 10 in ensemble
08:58  Expected score (should be close to zero): [-0.06124121  0.01884908]
08:58  Starting evaluation for estimator 2 / 10 in ensemble
08:58  Expected score (should be close to zero): [ 0.017485   -0.00885739]
08:58  Starting evaluation for estimator 3 / 10 in ensemble
08:58  Expected score (should be close to zero): [-0.03697696 -0.00906709]
08:58  Starting evaluation for estimator 4 / 10 in en

[[70052.82149736  2314.43994685]
 [ 2314.43994685  2515.29243761]]


In [15]:
means, covs = fisher_tight.calculate_fisher_information_full_detector(
    theta=[0.,0.],
    luminosity=300000.,
    model_file=model_dir + 'sally_ensemble_all_tight',
    unweighted_x_sample_file = sample_dir + 'test_tight/x_test.npy',
    ensemble_vote_expectation_weight=weight_settings
)

print(means[0])

for weight_filename, mean, cov in zip(weight_setting_filenames, means, covs):
    np.save(result_dir + 'information_mean_full_tight{}.npy'.format(weight_filename), mean)
    np.save(result_dir + 'information_covariance_full_tight{}.npy'.format(weight_filename), cov)
    

09:01  Loading ensemble setup from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_all_tight/ensemble.json
09:01  Found ensemble with 10 estimators and expectations [[ 0.12302641 -0.19173431]
 [-0.00147447  0.05780574]
 [-0.21044625 -0.10930192]
 [ 0.02109473  0.26448175]
 [-0.22535108  0.1274436 ]
 [ 0.08491807  0.13141175]
 [-0.20341797 -0.30981925]
 [-0.12194088 -0.20932615]
 [ 0.05521736  0.34293151]
 [-0.02142752 -0.00210372]]
09:01  Evaluating Fisher information for 10 estimators in ensemble
09:01  Starting evaluation for estimator 1 / 10 in ensemble
09:01  Expected score (should be close to zero): [ 0.12306172 -0.19178116]
09:01  Starting evaluation for estimator 2 / 10 in ensemble
09:01  Expected score (should be close to zero): [-0.00119139  0.05779046]
09:01  Starting evaluation for estimator 3 / 10 in ensemble
09:02  Expected score (should be close to zero): [-0.21248028 -0.10795032]
09:02  Starting evaluation for estimator 4 / 10

[[684.77828882   8.47579521]
 [  8.47579521 725.78114884]]


## Info in observables (ML)

In [ ]:
means, covs = fisher_tight.calculate_fisher_information_full_detector(
    theta=[0.,0.],
    luminosity=300000.,
    model_file=model_dir + 'sally_ensemble_resurrection_tight',
    unweighted_x_sample_file = sample_dir + 'test_tight/x_test.npy',
    ensemble_vote_expectation_weight=weight_settings
)

print(means[0])

for weight_filename, mean, cov in zip(weight_setting_filenames, means, covs):
    np.save(result_dir + 'information_mean_resurrection_tight{}.npy'.format(weight_filename), mean)
    np.save(result_dir + 'information_covariance_resurrection_tight{}.npy'.format(weight_filename), cov)
    

## Histogram info

In [18]:
filenames = ['met', 'ptl', 'pta', 'deltaphi_lv', 'deltaphi_la', 'phi_resurrection']
observables = ['et_miss', 'pt_l1', 'pt_a1', 'deltaphi_l1_met', 'deltaphi_l1_a1', 'phi_resurrection']
bins = 10

In [17]:
for filename, obs in zip(filenames, observables):
    info, cov = fisher_tight.calculate_fisher_information_hist1d(
                theta=[0.,0.],
                luminosity=300000.,
                observable=obs,
                nbins=bins,
                histrange=None,
                
           )
    
    print(filename)
    print(info)
    
    np.save(result_dir + 'information_histo_{}_tight.npy'.format(filename), info)
    

/Users/johannbrehmer/work/projects/madminer/madminer/madminer/fisherinformation.py:831: RuntimeWarning: divide by zero encountered in true_divide
  inv_sigma = np.nan_to_num(1. / sigma)  # Shape (n_events,)


met
[[ 4376.52039533 -2034.43695455]
 [-2034.43695455  5787.96448357]]
ptl
[[ 4093.52964251 -2229.84875968]
 [-2229.84875968  7538.10972165]]
pta
[[4116.01267786 -852.24541136]
 [-852.24541136 5523.03065043]]
deltaphi_lv
[[85151.63107271  -543.51339519]
 [ -543.51339519  6746.07081552]]
deltaphi_la
[[33464.74170304   812.0884575 ]
 [  812.0884575   3753.37771299]]


KeyboardInterrupt: 

In [20]:
for filename, obs in zip(filenames, observables):
    info = fisher_tight.calculate_fisher_information_hist1d(
                theta=[0.,0.],
                luminosity=300000.,
                observable=obs,
                nbins=bins,
                histrange=None
           )
    
    print(filename)
    print(info)
    
    np.save(result_dir + 'information_histo_{}_tight.npy'.format(filename), info)
    

/Users/johannbrehmer/work/projects/madminer/madminer/madminer/fisherinformation.py:831: RuntimeWarning: divide by zero encountered in true_divide
  inv_sigma = np.nan_to_num(1. / sigma)  # Shape (n_events,)


met
[[2505.31391203 -970.82420597]
 [-970.82420597 1592.81043509]]
ptl
[[ 3051.32837606 -1223.23590267]
 [-1223.23590267  2403.70158   ]]
pta
[[ 987.16364647 -105.72922911]
 [-105.72922911 1804.03113204]]
deltaphi_lv
[[77964.18467768  1361.26469788]
 [ 1361.26469788  2202.90702006]]
deltaphi_la
[[29474.08977038  1089.85040033]
 [ 1089.85040033  1524.3092301 ]]
phi_resurrection
[[3742.99517778 -601.92910942]
 [-601.92910942 1694.45983927]]
